<a href="https://colab.research.google.com/github/JoshOlu/data602/blob/main/week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.16

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as log_reg, Lasso, Ridge
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score
import statsmodels.api as sm

In [4]:
from google.colab import files
uploaded = files.upload()
boston_df = pd.read_csv("Boston.csv")
boston_df.head()

Saving Boston.csv to Boston (1).csv


,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [5]:
boston_df = boston_df.select_dtypes(include=np.number)
boston_df.dropna(how="any", inplace=True)
boston_df.reset_index(inplace=True, drop=True)
boston_df.head()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2


In [6]:
boston_df["crim"].median()

0.25651

In [7]:
boston_df["higher_crime"] = (boston_df["crim"] > boston_df["crim"].median()).astype(int)
boston_df.head()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,lstat,medv,higher_crime
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,0
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,0
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,0
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,0


In [8]:
def fit_log_reg(X, y):
    x_intercept = sm.add_constant(X)
    logit_stats = sm.Logit(y, x_intercept)
    results_stats = logit_stats.fit()

    y_pred_prob = results_stats.predict(x_intercept)
    y_pred = (y_pred_prob > 0.5).astype(int)

    accuracy = accuracy_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)
    class_report = classification_report(y, y_pred)

    return results_stats.summary(), accuracy, conf_matrix, class_report

subsets = [["rm", "age"],["rm", "tax"],["rm", "medv"],["indus", "rad"],["age", "nox"],["age", "medv"]]
target = boston_df["higher_crime"]

In [9]:
for sets in subsets:
    print(f"\nPredictor Subsets: {sets}")
    features = boston_df[sets]
    summary, accuracy, conf_matrix, class_report = fit_log_reg(features, target)
    print(summary)
    print(f"Accuracy: {accuracy}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(f"Classification Report:\n{class_report}")


Predictor Subsets: ['rm', 'age']
Optimization terminated successfully.
         Current function value: 0.471338
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           higher_crime   No. Observations:                  506
Model:                          Logit   Df Residuals:                      503
Method:                           MLE   Df Model:                            2
Date:                Fri, 04 Oct 2024   Pseudo R-squ.:                  0.3200
Time:                        03:16:18   Log-Likelihood:                -238.50
converged:                       True   LL-Null:                       -350.73
Covariance Type:            nonrobust   LLR p-value:                 1.806e-49
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.2433      1.177     -3.606      0.000      -6.550      -1

I used the following subset of predictors: ["rm", "age"],["rm", "tax"],["rm", "medv"],["indus", "rad"],["age", "nox"],["age", "medv"]

What I've found is that majority of the subsets with "age" have a p-value near zero, which could indicate that "age" has significance as a prefictor. Also, in these same subsets including "age," the positive coefficient suggests that older homes have a higher crime rate.

The subsets that include "medv" have a lower accuracy that the others, on average This could suggest that "medv" doesn't have enough significance to suggest an affect on higher crime rates.

The subsets that use "tax," "nos," "indus," and "rad" seem to have better accuracy with more significance (p-values < 0.05), which suggests that they can have an impact on increasing crime rates.

6.11.a

# 6.11.a

In [10]:
new_feature = boston_df.drop(columns=["crim", "higher_crime"])
new_target = boston_df["crim"]

scaler = StandardScaler()
scaled_feature = scaler.fit_transform(new_feature)

In [11]:
lasso = Lasso(alpha=0.1)
lasso.fit(scaled_feature, new_target)
lasso_target = lasso.predict(scaled_feature)

In [12]:
ridge = Ridge(alpha=0.1)
ridge.fit(scaled_feature, new_target)
ridge_target = ridge.predict(scaled_feature)

In [13]:
lasso_mse = mean_squared_error(new_target, lasso_target)
ridge_mse = mean_squared_error(new_target, ridge_target)
lasso_r2 = r2_score(new_target, lasso_target)
ridge_r2 = r2_score(new_target, ridge_target)

In [14]:
print(f"Lasso Regression: MSE = {lasso_mse:.4f}, R² = {lasso_r2:.4f}")
print(f"Ridge Regression: MSE = {ridge_mse:.4f}, R² = {ridge_r2:.4f}")

print("\nLasso Coefficients")
print(pd.Series(lasso.coef_, index=new_feature.columns))

print("\nRidge Coefficients")
print(pd.Series(ridge.coef_, index=new_feature.columns))

Lasso Regression: MSE = 41.0902, R² = 0.4435
Ridge Regression: MSE = 40.6273, R² = 0.4498

Lasso Coefficients
Unnamed: 0   -0.000000
zn            0.758115
indus        -0.312444
chas         -0.151333
nox          -0.409380
rm            0.148587
age           0.000000
dis          -1.294094
rad           4.521374
tax          -0.000000
ptratio      -0.307236
lstat         0.908827
medv         -1.349546
dtype: float64

Ridge Coefficients
Unnamed: 0   -0.263777
zn            1.091602
indus        -0.404313
chas         -0.204249
nox          -1.134973
rm            0.460361
age          -0.076295
dis          -2.157816
rad           5.436478
tax          -0.542575
ptratio      -0.652213
lstat         0.974647
medv         -2.035642
dtype: float64


What I found from using LASSO and Ridge regression is that ridge has a smaller mean squared error than LASSO. Although, they are fairly close and are overall low. This suggests the predictions are close to the actual value of crime rate per capita.

In terms of the coefficients, the higher coefficients (farthest from zero) have a high impact/significance on the target variable, being crime.

For both LASSO and Ridge, the features with the higher positive coefficients include zn and rad, indicating that as these predictors increase, crime rate tends to increase as well.

The features with the higher negative coefficients include nox, dis, and medv, indicating that as these predictors descrease, crime rate tends to decrease.